[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/lora_sd1.5_lcm_draw_colab.ipynb)

In [ ]:
!pip install -q diffusers transformers accelerate peft gradio==3.50.2
!pip install -q git+https://github.com/tencent-ailab/IP-Adapter einops
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

!mkdir /content/models
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors?download=true -O /content/models/ip-adapter_sd15.safetensors

!mkdir /content/image_encoder
!wget https://huggingface.co/h94/IP-Adapter/raw/main/models/image_encoder/config.json -O /content/image_encoder/config.json
!wget https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors?download=true -O /content/image_encoder/model.safetensors

import torch
from diffusers import StableDiffusionControlNetPipeline, DDIMScheduler, AutoencoderKL, ControlNetModel, LCMScheduler
from PIL import Image

vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to(dtype=torch.float16)
noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1e_sd15_tile", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
).to('cuda')
pipe.enable_xformers_memory_efficient_attention()

# pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
# pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
# pipe.fuse_lora()

from ip_adapter import IPAdapter
ip_model = IPAdapter(pipe, '/content/image_encoder', '/content/models/ip-adapter_sd15.safetensors', 'cuda')

import gradio as gr

def generate(prompt, input_image):
    image = ip_model.generate(pil_image=input_image, image=input_image, num_samples=1, num_inference_steps=30)[0]
    # image = pipe(prompt, image=input_image, num_inference_steps=30, guidance_scale=8.0).images[0]
    return image.resize((768, 768))

with gr.Blocks(title=f"Realtime Latent Consistency Model") as demo:
    with gr.Row():
      with gr.Column(scale=23):
          textbox = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat")

    with gr.Row(variant="default"):
        input_image = gr.Image(
            show_label=False,
            type="pil",
            tool="color-sketch",
            source="canvas",
            height=742,
            width=742,
            brush_radius=10.0,
        )
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=742,
            width=742,
            elem_id="output_image",
        )

    textbox.change(fn=generate, inputs=[textbox, input_image], outputs=[output_image], show_progress=False)
    input_image.change(fn=generate, inputs=[textbox, input_image], outputs=[output_image], show_progress=False)

demo.launch(inline=False, share=True, debug=True)